## Browsing HSC Footprints

This notebook tests new functionality in `lsst.display.firefly` for browsing catalogs, footprints and images.

In [ ]:
import numpy as np

In [ ]:
from lsst.daf.persistence import Butler
import lsst.afw.geom as afwGeom

In [ ]:
import lsst.afw.display as afwDisplay

In [ ]:
from lsst.display.firefly import footprints

This Butler repo has a small subset of `/datasets/hsc/repo/rerun/RC/w_2018_38/DM-15690/deepCoadd-results`

In [ ]:
butler = Butler('/home/vagrant/mnt/data/lsst2018-data/')

Define an ID for retrieving data

In [ ]:
dataId = dict(filter='HSC-R', tract=9813, patch='4,4')

Define a bounding box for a region of interest

In [ ]:
bbox = afwGeom.Box2I(corner=afwGeom.Point2I(16900, 18700),
                     dimensions=afwGeom.Extent2I(600,600))

Retrieve a catalog for the entire region

In [ ]:
measCat = butler.get('deepCoadd_meas', dataId=dataId)

Select only those records with pixel locations inside the bounding box

In [ ]:
cat_select = np.array([bbox.contains(afwGeom.Point2I(r.getX(), r.getY())) for r in measCat])
catalog_subset = measCat.subset(cat_select)

Set up the Firefly display. In this case we are running inside a Docker container and using a Firefly server on localhost.

In [ ]:
url = 'http://host.docker.internal:8080/firefly'
display1 = afwDisplay.Display(frame=1, backend='firefly', url=url,
                             name='footprint-test')

In [ ]:
display1.setMaskTransparency(90)

In [ ]:
display1.scale('asinh', 10, 80, unit='percent', Q=6)

Display the full image and overlay the footprints. 

In [ ]:
footprints.browse_footprints(catalog_subset, display1, dataId, butler)

Zoom to the center of the BoundingBox

In [ ]:
center = bbox.getBegin() + bbox.getDimensions()/2.0
display1.zoom(0.5, center[0], center[1])